# 資料處理

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, fbeta_score, precision_score, recall_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import ParameterGrid
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm.notebook import tqdm
import csv
from gensim.models.callbacks import CallbackAny2Vec
from itertools import groupby
from gensim.models import Word2Vec
import gensim
from pathlib import Path
import matplotlib.pyplot as plt
import os
import random
import collections
import pickle
from sklearn.preprocessing import MinMaxScaler
pd.options.display.max_columns = 999

In [2]:
PATH = Path('/home/hsien/110/sta/imdb-dataset/')

In [3]:
for dirname, _, filenames in os.walk(PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/hsien/110/sta/imdb-dataset/title.akas.tsv.gz
/home/hsien/110/sta/imdb-dataset/name.basics.tsv.gz
/home/hsien/110/sta/imdb-dataset/title.ratings.tsv.gz
/home/hsien/110/sta/imdb-dataset/title.principals.tsv.gz
/home/hsien/110/sta/imdb-dataset/title.basics.tsv.gz
/home/hsien/110/sta/imdb-dataset/title.basics.tsv/title.basics.tsv
/home/hsien/110/sta/imdb-dataset/title.ratings.tsv/title.ratings.tsv
/home/hsien/110/sta/imdb-dataset/name.basics.tsv/name.basics.tsv
/home/hsien/110/sta/imdb-dataset/title.principals.tsv/title.principals.tsv
/home/hsien/110/sta/imdb-dataset/title.akas.tsv/title.akas.tsv


# Load Data

In [4]:
def load_data(start_year, min_minutes, min_votes):
    title_basics = pd.read_csv(PATH / 'title.basics.tsv' / 'title.basics.tsv', sep='\t', dtype={"startYear": str})
    title_ratings = pd.read_csv(PATH / 'title.ratings.tsv'/ 'title.ratings.tsv', sep='\t')
    title_basics.genres = title_basics.genres.apply(
                        lambda x: x.split(',') if ((type(x)!=float) & (x!=r'\N')) else ['no_genre'])

    title_basics.runtimeMinutes = (
     title_basics.runtimeMinutes.apply(lambda x: np.nan if not x.isdigit() else x).astype(float)
                                )

    movies = pd.merge(title_basics, title_ratings, on='tconst', how='left')
    movies['startYear'] = movies['startYear'].apply(lambda x: np.nan if x == r'\N' else int(x))
    
    # MY CONDITIONS:
    movies = movies[movies.startYear > start_year].dropna(subset=['averageRating'])
    movies = movies[movies.runtimeMinutes >= min_minutes]
    movies = movies[movies.numVotes>=min_votes]
    return movies

In [5]:
movies = load_data(start_year = 1960, min_minutes = 60, min_votes = 15)
movies['startYear'] = movies['startYear'].astype('int64')
movies['numVotes'] = movies['numVotes'].astype('int64')

In [6]:
movies.sort_values('startYear', inplace=True)

In [7]:
movies.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
144923,tt0149295,movie,Il trionfo di Maciste,Il trionfo di Maciste,0,1961,\N,88.0,"[Action, Adventure, Drama]",4.2,84
720606,tt0743141,tvEpisode,The Jed Polke Story,The Jed Polke Story,0,1961,\N,60.0,[Western],8.2,30
55055,tt0056099,movie,Information Received,Information Received,0,1961,\N,77.0,[Crime],5.8,15
263357,tt0274612,movie,The Red Vase,Kizil vazo,0,1961,\N,97.0,[Drama],5.9,18
720609,tt0743144,tvEpisode,The Jenna Douglas Story,The Jenna Douglas Story,0,1961,\N,60.0,[Western],7.7,21


In [8]:
# with open('final_pkfile/original_df.pk', 'wb') as f:
#     pickle.dump(movies, f)

# 處理cast跟crew

In [9]:
principals = pd.read_csv(PATH /'title.principals.tsv'/'title.principals.tsv',delimiter="\t")

We divide our "principals" dataset into "cast" (actors and actresses) and the rest.
The idea is to add to our Movies dataframe two columns: one with a list containing all actors/actresses in the movie, and another including all other crew members who were involved in the movie.

In [10]:
cast = principals[principals['category'].isin(['actor', 'actress'])]
crew = principals[~principals['category'].isin(['actor', 'actress'])]

In [11]:
ordered_cast = pd.DataFrame(cast.groupby(by='tconst').apply(lambda x: [x for x in x['nconst']]))
ordered_cast = ordered_cast.reset_index().rename(columns={0: 'cast'})
# ordered_cast.head(2)

In [12]:
ordered_crew = pd.DataFrame(crew.groupby(by='tconst').apply(lambda x: [x for x in x['nconst']]))
ordered_crew = ordered_crew.reset_index().rename(columns={0: 'crew'})
# ordered_crew.head(2)

In [13]:
# with open('ordered_cast.pk', 'rb') as f:
#     ordered_cast = pickle.load(f)
# with open('ordered_crew.pk', 'rb') as f:
#     ordered_crew = pickle.load(f)

Now we add this information to our movies data.

In [14]:
movies = pd.merge(movies, ordered_cast, on='tconst', how='left').merge(ordered_crew, on='tconst', how='left')

# If it went right, we should have no duplicated movies (tconst)
# movies.duplicated('tconst').any()

In [15]:
# with open('final_pkfile/ordered_cast.pk', 'wb') as f:
#     pickle.dump(ordered_cast, f)
# with open('final_pkfile/ordered_crew.pk', 'wb') as f:
#     pickle.dump(ordered_crew, f)

In [16]:
# CHECK: Back to The Future should have Fox and C. Lloyd within Cast, and Robert Zemeckis within Crew.
movies[movies['tconst']=='tt0088763']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,cast,crew
43731,tt0088763,movie,Back to the Future,Back to the Future,0,1985,\N,116.0,"[Adventure, Comedy, Sci-Fi]",8.5,965539,"[nm0000150, nm0000502, nm0000670, nm0000417]","[nm0004637, nm0000709, nm0301826, nm0134635, n..."


In [17]:
movies['cast'].isna().sum()

23068

In [18]:
movies['crew'].isna().sum()

6078

In [19]:
movies[movies['cast'].isna()].head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,cast,crew
80,tt0268567,tvEpisode,The Real West,The Real West,0,1961,\N,60.0,[Documentary],8.0,27,NaN,"[nm0000011, nm1602478, nm0718448, nm0072021, n..."
94,tt0196879,movie,The Human Pyramid,La pyramide humaine,0,1961,\N,90.0,[Drama],7.7,264,NaN,"[nm0950391, nm0050913, nm1668784, nm1682691, n..."


In [20]:
# with open('final_pkfile/movies.pk', 'wb') as f:
#     pickle.dump(movies, f)

# with open('final_pkfile/movies.pk', 'rb') as f:
#     movies = pickle.load(f)

In [21]:
movies['cast'].fillna('Unknown', inplace=True)
movies['crew'].fillna('Unknown', inplace=True)

# 整理資料格式

In [22]:
imdb_df = movies
imdb_df['change_name'] = np.where(imdb_df['primaryTitle'] != imdb_df['originalTitle'], 1, 0)
imdb_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,cast,crew,change_name
0,tt0149295,movie,Il trionfo di Maciste,Il trionfo di Maciste,0,1961,\N,88.0,"[Action, Adventure, Drama]",4.2,84,"[nm0606714, nm0138900, nm0090960, nm0266945]","[nm0733504, nm0090557, nm0220383, nm0778644, n...",0
1,tt0743141,tvEpisode,The Jed Polke Story,The Jed Polke Story,0,1961,\N,60.0,[Western],8.2,30,"[nm0570615, nm0395667, nm0934211, nm0569795]","[nm0658241, nm0900972, nm0314558, nm0160330, n...",0
2,tt0056099,movie,Information Received,Information Received,0,1961,\N,77.0,[Crime],5.8,15,"[nm0785923, nm0843213, nm0045968, nm0880956]","[nm0805593, nm0528790, nm0558435, nm0753132, n...",0
3,tt0274612,movie,The Red Vase,Kizil vazo,0,1961,\N,97.0,[Drama],5.9,18,"[nm0234232, nm0037593, nm0038232, nm0040320]","[nm0277266, nm0947998, nm0147924, nm0879609, n...",1
4,tt0743144,tvEpisode,The Jenna Douglas Story,The Jenna Douglas Story,0,1961,\N,60.0,[Western],7.7,21,"[nm0570615, nm0395667, nm0427700, nm0527017]","[nm0277660, nm0900972, nm0569950, nm0642683, n...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203516,tt7159616,movie,Puppy Swap Love Unleashed,Puppy Swap Love Unleashed,0,2019,\N,90.0,[Family],5.8,75,"[nm2963701, nm0385032, nm0005269, nm4010544]","[nm5416484, nm4433696, nm0437490, nm1788026, n...",0
203517,tt7158904,movie,Blood Myth,Blood Myth,0,2019,\N,81.0,[Horror],3.0,123,"[nm6920118, nm5139477, nm3892357, nm2184066]","[nm4105518, nm6920125, nm6920123, nm6920120]",0
203518,tt7156898,movie,County Lines,County Lines,0,2019,\N,90.0,[Drama],8.4,24,"[nm6170168, nm0534772, nm8519744, nm1599744]","[nm1147435, nm2207245, nm2540589, nm0110796, n...",0
203519,tt7216068,movie,See You Soon,See You Soon,0,2019,\N,107.0,"[Drama, Romance]",4.7,265,"[nm0539421, nm0000172, nm2553987, nm5014360, n...","[nm2550041, nm9191743, nm9177847, nm0534673, n...",0


In [23]:
imdb_df.shape

(203521, 14)

# One-hot encoding

### one label encoding (titleType)

In [24]:
pd.get_dummies(imdb_df['titleType']).head(2)

,movie,short,tvEpisode,tvMiniSeries,tvMovie,tvSeries,tvShort,tvSpecial,video,videoGame
0,1,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0


In [25]:
imdb_df = pd.concat((imdb_df, pd.get_dummies(imdb_df['titleType'])), axis=1)

In [26]:
imdb_df.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,cast,crew,change_name,movie,short,tvEpisode,tvMiniSeries,tvMovie,tvSeries,tvShort,tvSpecial,video,videoGame
0,tt0149295,movie,Il trionfo di Maciste,Il trionfo di Maciste,0,1961,\N,88.0,"[Action, Adventure, Drama]",4.2,84,"[nm0606714, nm0138900, nm0090960, nm0266945]","[nm0733504, nm0090557, nm0220383, nm0778644, n...",0,1,0,0,0,0,0,0,0,0,0
1,tt0743141,tvEpisode,The Jed Polke Story,The Jed Polke Story,0,1961,\N,60.0,[Western],8.2,30,"[nm0570615, nm0395667, nm0934211, nm0569795]","[nm0658241, nm0900972, nm0314558, nm0160330, n...",0,0,0,1,0,0,0,0,0,0,0


### multilabel encoding (genres, cast, crew)

#### 處理genres

In [27]:
imdb_df['genres'].str.join(sep='*').str.get_dummies(sep='*').head(2)

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,no_genre
0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [28]:
imdb_df = pd.concat((imdb_df, imdb_df['genres'].str.join(sep='*').str.get_dummies(sep='*')), axis = 1)

##### 刪除['no_genre'] column

In [29]:
imdb_df.drop(columns=['no_genre'], inplace=True)

#### 處理cast跟crew
- 用multilabel encoding要400G、576G
- 使用word2vec

寫成檔案方便沒有one-hot時需要

In [30]:
# with open('final_pkfile/imdb_df_genres_title_cgname.pk', 'wb') as f:
#     pickle.dump(imdb_df, f)

看下前五個cast

In [31]:
list(imdb_df['cast'][:5])

[['nm0606714', 'nm0138900', 'nm0090960', 'nm0266945'],
 ['nm0570615', 'nm0395667', 'nm0934211', 'nm0569795'],
 ['nm0785923', 'nm0843213', 'nm0045968', 'nm0880956'],
 ['nm0234232', 'nm0037593', 'nm0038232', 'nm0040320'],
 ['nm0570615', 'nm0395667', 'nm0427700', 'nm0527017']]

##### MultiLabel Binarizer

 沒特別用(sparse_output=True)

In [32]:
# # Unable to allocate 406. GiB for an array with shape (203521, 267796) and data type int64
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()
# cast_arr = mlb.fit_transform(list(imdb_df['cast']))

In [33]:
# # Unable to allocate 406. GiB for an array with shape (203521, 267796) and data type int64
# imdb_df = pd.concat((imdb_df, imdb_df['cast'].str.join(sep='*').str.get_dummies(sep='*')), axis = 1)

In [34]:
# # Unable to allocate 576. GiB for an array with shape (203521, 379684) and data type int64
# imdb_df = pd.concat((imdb_df, imdb_df['crew'].str.join(sep='*').str.get_dummies(sep='*')), axis = 1)

用(sparse_output=True) <br>
但我下面預測就跑不動了

In [35]:
# # Unable to allocate 406. GiB for an array with shape (203521, 267796) and data type int64
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer(sparse_output=True)
# crew_arr = mlb.fit_transform(list(imdb_df['crew']))

##### 放棄上面方法，改選少一點演員

In [36]:
cast_list = list(i for x in imdb_df['cast'] for i in x)
crew_list = list(i for x in imdb_df['crew'] for i in x)

In [37]:
cast_counter=collections.Counter(cast_list)
crew_counter=collections.Counter(crew_list)

In [38]:
del cast_counter['U']
del cast_counter['k']
del cast_counter['n']
del cast_counter['o']
del cast_counter['w']
del crew_counter['U']
del crew_counter['k']
del crew_counter['n']
del crew_counter['o']
del crew_counter['w']

In [39]:
print(len(cast_list))
print(len(crew_list))

953266
1144446


##### 超過50次的演員有幾位

In [40]:
cast_minthreshold = 50
cast_subdict = {x: count for x, count in cast_counter.items() if count >= cast_minthreshold}
len(cast_subdict)

1121

##### 超過50次的工作人員有幾位

In [41]:
crew_minthreshold = 50
crew_subdict = {x: count for x, count in crew_counter.items() if count >= crew_minthreshold}
len(crew_subdict)

1091

##### 加入dataframe

In [42]:
cast_data = {}
for elem in cast_subdict.keys():
    cast_data[elem] = np.array(list(elem in x for x in imdb_df['cast']), dtype=int)

In [43]:
len(cast_data)

1121

In [44]:
crew_data = {}
for elem in crew_subdict.keys():
    crew_data[elem] = np.array(list(elem in x for x in imdb_df['crew']), dtype=int)

In [45]:
len(crew_data)

1091

In [46]:
cast_data_df = pd.DataFrame.from_dict(cast_data,orient='index').transpose()

In [47]:
crew_data_df = pd.DataFrame.from_dict(crew_data,orient='index').transpose()

In [48]:
# with open('final_pkfile/cast_onehot_df.pk', 'wb') as f:
#     pickle.dump(cast_data_df, f)

In [49]:
# with open('final_pkfile/crew_onehot_df.pk', 'wb') as f:
#     pickle.dump(crew_data_df, f)

In [50]:
cast_data_df.shape

(203521, 1121)

In [51]:
crew_data_df.shape

(203521, 1091)

In [52]:
cast_data_df

,nm0570615,nm0934211,nm0569795,nm0588225,nm0731490,nm0088779,nm0001446,nm0001296,nm0721073,nm0000890,nm0000308,nm0000527,nm0951019,nm0814734,nm0019699,nm0000052,nm0762248,nm0893099,nm0001954,nm0000125,nm0660362,nm0365835,nm0808774,nm0298333,nm0000790,nm0915840,nm0832065,nm0086469,nm0320883,nm0066494,nm0701500,nm0001588,nm0088396,nm0819825,nm0396136,nm0639152,nm0865575,nm0007023,nm0313443,nm0108400,nm0000901,nm0731772,nm0813961,nm0704719,nm0000314,nm0000587,nm0000869,nm0001981,nm0391800,nm0814773,nm0619938,nm0001313,nm0000489,nm0634159,nm0723623,nm0000018,nm0002002,nm0001800,nm0001583,nm0418148,nm0000063,nm0001725,nm0001262,nm0121826,nm0573024,nm0068772,nm0540487,nm0001682,nm0956544,nm0809783,nm0519160,nm0126318,nm0703919,nm0514059,nm0000393,nm0004434,nm0000994,nm0354853,nm0394438,nm0848251,nm0542063,nm0001088,nm0001430,nm0000842,nm0409869,nm0000549,nm0694619,nm0001171,nm0448906,nm0001017,nm0240887,nm0106539,nm0734000,nm0301556,nm0487108,nm0000638,nm0001816,nm0001012,nm0004429,nm0237244,nm0907636,nm0000995,nm0565982,nm0000336,nm0000328,nm0000558,nm0957372,nm0798328,nm0006573,nm0593192,nm0587401,nm0000919,nm0001715,nm0001115,nm0134201,nm0912001,nm0810342,nm0001421,nm0027323,nm0533891,nm0191685,nm0001417,nm0943978,nm0759685,nm0000728,nm0593369,nm0320760,nm0674742,nm0926183,nm0001768,nm0000053,nm0001369,nm0001687,nm0000577,nm0001657,nm0001673,nm0002094,nm0389537,nm0001258,nm0518728,nm0004462,nm0289960,nm0409049,nm0903866,nm0001544,nm0001637,nm0144045,nm0001141,nm0000032,nm0001428,nm0226947,nm0154164,nm0607558,nm0768334,nm0273178,nm0001500,nm0001128,nm0000348,nm0385757,nm0001884,nm0341431,nm0001665,nm0000804,nm0001822,nm0001321,nm0564724,nm0466327,nm0312575,nm0000051,nm0480942,nm0255678,nm0105475,nm0000454,nm0603402,nm0281661,nm0000142,nm0109495,nm0048371,nm0006369,nm7390393,nm0015360,nm0001777,nm0347656,nm0000380,nm0000024,nm0001132,nm0619309,nm0000602,nm0005279,nm0001458,nm0002011,nm0164766,nm0238628,nm0000608,nm0001066,nm0758692,nm0193411,nm0695199,nm0015391,nm0419275,nm0001136,nm0001991,nm0662240,nm0104507,nm0047962,nm0654843,nm0574534,nm0943837,nm0001859,nm0547339,nm0000366,nm0001703,nm0785664,nm0002145,nm0472545,nm0000564,nm0114543,nm0138405,nm0000090,nm0001157,nm0000268,nm0001445,nm0114422,nm0095522,nm0913993,nm0253020,nm0475610,nm0485542,nm0852076,nm0001699,nm0001001,nm0846616,nm0336335,nm0001719,nm0058001,nm0176684,nm0662981,nm0294229,nm0001831,nm0000799,nm0005162,nm0125627,nm0462607,nm0681566,nm0929934,nm0873757,nm0861943,nm0000621,nm0001186,nm0847264,nm0303168,nm0001813,nm0000991,nm0000603,nm0000993,nm0001077,nm0004467,nm0000329,nm0040910,nm0352032,nm0822034,nm0001228,nm0002016,nm0289038,nm0000978,nm0761100,nm0569000,nm0035067,nm0237771,nm0921202,nm0001522,nm0024695,nm0086323,nm0001021,nm0385976,nm0041281,nm0484376,nm0357745,nm0001306,nm0001626,nm0000323,nm0004569,nm0903110,nm0001702,nm0001285,nm0527032,nm0858186,nm0148041,nm0935282,nm0005212,nm0001648,nm0159159,nm0001036,nm0001070,nm0000977,nm0267232,nm0646037,nm0712022,nm0316284,nm0000516,nm0002067,nm0553014,nm0187337,nm0004833,nm0946179,nm0764134,nm0329730,nm0003909,nm0516316,nm0002663,nm0321779,nm0000432,nm0004857,nm0140649,nm0000273,nm0474876,nm0626259,nm0683986,nm0703687,nm0000559,nm0001358,nm0246150,nm0001016,nm0026392,nm0001347,nm0000453,nm0004435,nm0313519,nm0170696,nm0580886,nm0000828,nm0606569,nm0123201,nm0201688,nm0000640,nm0001868,nm0227708,nm0001449,nm0527002,nm0004486,nm0000172,nm0133597,nm0296260,nm0001513,nm0559837,nm0420090,nm0201000,nm0001917,nm0817881,nm0000398,nm0756634,nm0045119,nm0000981,nm0000302,nm0000685,nm0001159,nm0000947,nm0802374,nm0593285,nm0062420,nm0000134,nm0000661,nm0001821,nm0005152,nm0028872,nm0004334,nm0271763,nm0815800,nm0903212,nm0784884,nm0494204,nm0000457,nm0000164,nm0473314,nm0000690,nm0000532,nm0004051,nm0000945,nm0644680,nm0004564,nm0096509,nm0777589,nm0389687,nm0001233,nm0614101,nm0755364,nm0155291,nm0936403,nm0897715,nm0156955,nm0001832,nm0000889,nm0574468,nm0001796,nm0001283,nm0862479,nm0000545,nm0000476,nm0001941,nm0001424

In [53]:
# with open('imdb_df.pk', 'wb') as f:
#     pickle.dump(imdb_df, f)

read

In [54]:
# with open('imdb_df.pk', 'rb') as f:
#     imdb_df = pickle.load(f)

### 刪除['originalTitle', 'endYear', 'titleType', 'genres', 'cast', 'crew']